In [ ]:
path = 'https://raw.githubusercontent.com/umangkejriwal1122/Machine-Learning/master/Data%20Sets/covid_19_clean_complete.csv'

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px     ### potting data on world map
import matplotlib.pyplot as plt

# pip install cython
# pip install pystan
# pip install fbprohet

In [ ]:
df = pd.read_csv(path)
df.head()

In [ ]:
df.info()

In [ ]:
df.drop(['Province/State'],axis=1,inplace = True)
df.head()

In [ ]:
df = df.rename(columns={"Country/Region" : 'country'})

In [ ]:
df.head()

In [ ]:
active = df['Confirmed'] - df['Recovered'] - df['Deaths']
active.tail()

In [ ]:
# concatenate alternative
df['Active'] = active
df.tail()

In [ ]:
# 
df.info()

In [ ]:
### Converting the Data Time object to datetime64[ns] format
df['Date'] = pd.to_datetime(df['Date'])
df.info()

In [ ]:
# Filter the data on the basis of the latest date
# top = df[df['Date'] == '2020-04-28']
top = df[df['Date'] == df['Date'].max()]
top.head()

In [ ]:
# Group on basis of the country(for all the dates) and display the cumulative 'Confirmed','Active','Recovered','Deaths'
# Resetting index since the country indented to show the group by "Basis"
# Also make country again as index since it's lost after grouping
# In case of accidental duplicates,groupby eliminates those
world = top.groupby('country')['Confirmed','Active','Recovered','Deaths'].sum().reset_index()
world.head()

In [ ]:
### Plot on the World Map((Active Cases))
figure = px.choropleth(world,locations='country',locationmode='country names',color='Active',
                       color_continuous_scale='Peach', range_color=[1,1000],
                       title = "Countries with Active Cases",hover_name = 'country')
figure.show()

In [ ]:
### Plot on the World Map((Active Cases))
figure = px.choropleth(world,locations='country',locationmode='country names',color='Deaths',
                       color_continuous_scale='darkmint', range_color=[1,10000],
                       title = "Countries with Deaths",hover_name = 'country')
figure.show()

In [ ]:
#### Plot WorldWide Confirmed Cases vs Date
## Add all the confimred cases based on date
total_confirmed_cases = df.groupby('Date')['Date','Confirmed'].sum().reset_index()
total_death_cases = df.groupby('Date')['Date','Deaths'].sum().reset_index()
total_confirmed_cases.tail()

In [ ]:
# Dates are overlapping and points are not clear so will have to modifying the plotting paramters
# pixel Dimensions
plt.figure(figsize=(16,10)) 

# rotate date/time by 90*, set fontsize = 10
plt.xticks(rotation = 90,fontsize = 10 ) 

# Extract only the date from date-time
sns.pointplot(total_confirmed_cases['Date'].dt.date,total_confirmed_cases['Confirmed'],color='red')
sns.pointplot(total_confirmed_cases['Date'].dt.date,total_death_cases['Deaths'],color='blue')

# Plotting the axes titles
plt.xlabel("Dates")
plt.ylabel("Total Confirmed Cases")

plt.show()

In [ ]:
# Dates are overlapping and points are not clear so will have to modifying the plotting paramters
# pixel Dimensions
plt.figure(figsize=(16,10)) 

# rotate date/time by 90*, set fontsize = 10
plt.xticks(rotation = 90,fontsize = 10 ) 

# Extract only the date from date-time
# sns.pointplot(total_confirmed_cases['Date'].dt.date,total_confirmed_cases['Confirmed'],color='red')
# sns.pointplot(total_confirmed_cases['Date'].dt.date,total_death_cases['Deaths'],color='blue')
plt.plot(total_confirmed_cases['Date'].dt.date,total_confirmed_cases['Confirmed'],color='red',label='Confirmed')
plt.plot(total_death_cases['Date'].dt.date,total_death_cases['Deaths'],color='blue',label='Deaths')

# Plotting the axes titles
plt.xlabel("Dates")
plt.ylabel("Total Deaths and Confirmed Cases")
plt.legend(loc=2)
plt.show()

In [ ]:
### Top 20 Countries having the most number of the Confirmed Cases
### Grouping based on country and sorting based on cumulative descending Confirmed cases
top_confirmed = top.groupby('country')['Confirmed'].sum().sort_values(ascending=False).reset_index()
top_confirmed.head()

In [ ]:
top_20 = top_confirmed.head(20)
print(top_20)

In [ ]:
### PLotting the data in the form of Bar Graph
plt.figure(figsize=(12,10))
sns.barplot(top_20['Confirmed'],top_20['country'],label='Confirmed') # barplot
plt.plot(top_20['Confirmed'],top_20['country'])  #lineplot
plt.scatter(top_20['Confirmed'],top_20['country']) #scatterplot
plt.legend(loc=4)  # loc = 4 (means 4th Quadrant)
plt.show()

In [ ]:
#### 

x= [1,2,3,4,5]
y=[10,20,20,30,40]
plt.bar(x,y,color = 'red')
plt.show()

In [ ]:
#### Make the data frames for some countries
## China 
china = df[df['country'] == 'China']
china = china.groupby('Date')['Active','Confirmed','Recovered','Deaths'].sum().reset_index()
china.head()

In [ ]:
# Italy
italy = df[df['country'] == 'Italy']
italy = italy.groupby('Date')['Active','Confirmed','Recovered','Deaths'].sum().reset_index()
italy.head()

In [ ]:
# India
india = df[df['country'] == 'India']
india = india.groupby('Date')['Active','Confirmed','Recovered','Deaths'].sum().reset_index()
india.head()

In [ ]:
# US
us = df[df['country'] == 'US']
us = us.groupby('Date')['Active','Confirmed','Recovered','Deaths'].sum().reset_index()
us.head()

In [ ]:
# Germany
germany = df[df['country'] == 'Germany']
germany = germany.groupby('Date')['Active','Confirmed','Recovered','Deaths'].sum().reset_index()
germany.head()

In [ ]:
# Taking the serial no on the X-axis
# Plotting all the line plots
plt.figure(figsize=(10,8))
plt.plot(china.index,china['Recovered'],color = 'red',label='China')
plt.plot(us.index,us['Recovered'],color = 'blue',label='US')
plt.plot(germany.index,germany['Recovered'],color = 'yellow',label='Germany')
plt.plot(india.index,india['Recovered'],color = 'green',label='India')
plt.plot(italy.index,italy['Recovered'],color = 'black',label='Italy')

# Plotting all the scatter plots
# Changing the marker style and size
# Remove labels so they don't appear again
plt.scatter(china.index,china['Recovered'],color = 'red',marker = "." , s = 200)
plt.scatter(us.index,us['Recovered'],color = 'blue')
plt.scatter(germany.index,germany['Recovered'],color = 'yellow')
plt.scatter(india.index,india['Recovered'],color = 'green')
plt.scatter(italy.index,italy['Recovered'],color = 'black')

plt.legend(loc = 2)
plt.show()

In [ ]:
### Forecasting of Time Series 

In [ ]:
### Using library Fbprophet - Time Series Analysis(Forecasting) - Created by Facebook
### Using ARIMA (ARIMA is an acronym that stands for AutoRegressive Integrated Moving Average.)

### Forcasting for confirmed cases

In [ ]:
# Fbprophet is the complete library
# Prophet is the name of class
from fbprophet import Prophet

In [ ]:
# Checking the date-time format
df.info()

In [ ]:
df.head()

In [ ]:
# Grouping by the date column
confirmed = df.groupby('Date')['Confirmed'].sum().reset_index()
confirmed.head()

In [ ]:
### According to the Fbprophet - Date Column name should be 'ds' and the data name should be 'y'
confirmed = confirmed.rename(columns={"Date" : "ds" , "Confirmed" : "y"})
confirmed.head()

In [ ]:
#### Building the model
## model will try its best to give 95 % accurate result (confidence interval in statistics)
## 5 % error is allowed
model = Prophet(interval_width=0.95)

In [ ]:
### Train the model
## In forecasting (unlike eg Linear Regreesion) , we pass entire data
model.fit(confirmed)
# Ignore the warning

In [ ]:
# We are forecasting for the next 7 days
future_dates = model.make_future_dataframe(periods=7)
future_dates.tail(7)

In [ ]:
# Predicting based on the trained model
forecast = model.predict(future_dates)
print(forecast)

In [ ]:
# Giving the range of values for forecast for the next few days
forecast[['ds','yhat','yhat_lower','yhat_upper']].tail(10)

In [ ]:
### Plot the data
## Dots = Actual value, Blue line = forcasted Value  
confirmed_plot = model.plot(forecast)

In [ ]:
### Plot Weekly Data
## People are going out on the weekends,that's why the weekends have higher rates
confirmed_plot_weekly = model.plot_components(forecast)

In [ ]:
### For India

In [ ]:
from fbprophet import Prophet

df_india = df[df['country']=='India']
df_india.head()


# Grouping by the date column
confirmed = df_india.groupby('Date')['Confirmed'].sum().reset_index()
confirmed.head()

### According to the Fbprophet - Date Column name should be 'ds' and the data name should be 'y'
confirmed = confirmed.rename(columns={"Date" : "ds" , "Confirmed" : "y"})
confirmed.head()

#### Building the model
## model will try its best to give 95 % accurate result (confidence interval in statistics)
## 5 % error is allowed
model = Prophet(interval_width=0.95)

### Train the model
## In forecasting (unlike eg Linear Regreesion) , we pass entire data
model.fit(confirmed)
# Ignore the warning

# We are forecasting for the next 7 days
future_dates = model.make_future_dataframe(periods=7)
future_dates.tail(7)

# Predicting based on the trained model
forecast = model.predict(future_dates)
print(forecast)

# Giving the range of values for forecast for the next few days
forecast[['ds','yhat','yhat_lower','yhat_upper']].tail(10)

### Plot the data
## Dots = Actual value, Blue line = forcasted Value  
confirmed_plot = model.plot(forecast)

print()

### Plot Weekly Data
## People are going out on the weekends,that's why the weekends have higher rates
confirmed_plot_weekly = model.plot_components(forecast)

In [ ]:
# More information on
# https://facebook.github.io/prophet/docs/diagnostics.html
